# Training the tensorflow model on Google Cloud AI Platform

In [ ]:
!sudo chown -R jupyter:jupyter /home/jupyter/Breast-Cancer-Detection

In [ ]:
import os

In [ ]:
PROJECT = 'vernal-buffer-285906'
REGION = 'us-west1'
BUCKET = 'vernal-buffer-285906'

In [ ]:
os.environ['BUCKET'] = BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION
os.environ['TFVERSION'] = '2.3'

In [ ]:
%%bash
gcloud config set project $PROJECT
gcloud config set compute/region $REGION

In [56]:
# os.environ['TRAIN_DIR'] = "/home/jupyter/Breast-Cancer-Detection/data"
# os.environ['OUTPUT_DIR'] = "/home/jupyter/Breast-Cancer-Detection/model_results"

os.environ['TRAIN_DIR'] = 'gs://vernal-buffer-285906/test_cnn'
os.environ['OUTPUT_DIR'] = 'gs://vernal-buffer-285906/model_results'

In [59]:
%%bash
export PYTHONPATH=${PYTHONPATH}:${PWD}/tf_trainer
#Train using python
python -m trainer.task \
 --input-dir=$TRAIN_DIR \
 --output-dir=$OUTPUT_DIR

hey hey hey
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 300, 300, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 150, 150, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 150, 150, 64)      18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 75, 75, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 75, 75, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 37, 37, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 37, 37, 

2021-02-21 16:23:50.911589: I tensorflow/core/platform/profile_utils/cpu_utils.cc:104] CPU Frequency: 2200140000 Hz
2021-02-21 16:23:50.912229: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55f35b4a6560 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2021-02-21 16:23:50.912365: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2021-02-21 16:23:50.912537: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
2021-02-21 16:23:51.882022: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session started.
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/opt/conda/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/jup

CalledProcessError: Command 'b'export PYTHONPATH=${PYTHONPATH}:${PWD}/tf_trainer\n#Train using python\npython -m trainer.task \\\n --input-dir=$TRAIN_DIR \\\n --output-dir=$OUTPUT_DIR\n'' returned non-zero exit status 1.

In [57]:
%%bash
export PYTHONPATH=${PYTHONPATH}:${PWD}/tf_trainer
    
OUTDIR=$OUTPUT_DIR
JOBNAME=breastcancer_$(date -u +%y%m%d_%H%M%S)
echo $OUTDIR $REGION $JOBNAME
gcloud ai-platform jobs submit training $JOBNAME \
  --region=$REGION \
  --module-name=trainer.task \
  --package-path=$(pwd)/tf_trainer/trainer \
  --job-dir=$OUTDIR \
  --staging-bucket=gs://$BUCKET \
  --scale-tier=BASIC \
  --runtime-version=2.3 \
  --python-version=3.7 \
  -- \
  --bucket=gs://$BUCKET \
  --output-dir=$OUTDIR \
  --input-dir=$TRAIN_DIR

gs://vernal-buffer-285906/model_results us-west1 breastcancer_210221_144033
jobId: breastcancer_210221_144033
state: QUEUED


Job [breastcancer_210221_144033] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ai-platform jobs describe breastcancer_210221_144033

or continue streaming the logs with the command

  $ gcloud ai-platform jobs stream-logs breastcancer_210221_144033
